# Loading the data
Make sure the CSV files provided by Zindi are in the same folder as this notebook. If it's running on Google Colab, you can use the 'files' tab on the left to upload them. We load the training data from Train.csv, and print out the variable descriptions here for easy reference. 

In [1]:
import pandas as pd
import numpy as np
import os, random, math, glob
from IPython.display import Image as IM
from IPython.display import clear_output
from matplotlib import pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = [16, 10]
import seaborn as sns
from sklearn.model_selection import train_test_split
from scipy.stats import boxcox

In [2]:
df = pd.read_csv('data/Train.csv')
df.head()

,ward,total_households,total_individuals,target,dw_00,dw_01,dw_02,dw_03,dw_04,dw_05,...,pw_03,pw_04,pw_05,pw_06,pw_07,pw_08,ADM4_PCODE,lat,lon,NL
0,41601001: Ward 1,1674.45058,5888.20750,16.773757,0.933841,0.000846,0.005490,0.000676,0.000000,0.001372,...,0.002848,0.007537,0.000000,0.012928,0,0,ZA4161001,-29.682270,24.734743,0.292039
1,41601002: Ward 2,1736.99230,6735.33812,21.496661,0.696940,0.001253,0.004402,0.000000,0.002301,0.001323,...,0.014566,0.057127,0.019092,0.004131,0,0,ZA4161002,-29.119311,24.757737,3.207775
2,41601003: Ward 3,2403.57591,7273.04995,10.931425,0.810545,0.004517,0.008891,0.003986,0.007735,0.000956,...,0.057560,0.010358,0.001421,0.040881,0,0,ZA4161003,-29.142276,25.094093,0.000000
3,41601004: Ward 4,1740.78737,5734.49046,23.119257,0.659914,0.000000,0.006129,0.000000,0.000813,0.037245,...,0.000000,0.000669,0.000000,0.005011,0,0,ZA4161004,-29.372052,24.942867,2.038778
4,41601005: Ward 5,1730.51451,6657.23835,13.652252,0.950575,0.000655,0.001473,0.000598,0.006999,0.000818,...,0.004859,0.001290,0.000673,0.017629,0,0,ZA4161005,-29.409381,25.290165,0.000000


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2822 entries, 0 to 2821
Data columns (total 63 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   ward               2822 non-null   object 
 1   total_households   2822 non-null   float64
 2   total_individuals  2822 non-null   float64
 3   target             2822 non-null   float64
 4   dw_00              2822 non-null   float64
 5   dw_01              2822 non-null   float64
 6   dw_02              2822 non-null   float64
 7   dw_03              2822 non-null   float64
 8   dw_04              2822 non-null   float64
 9   dw_05              2822 non-null   float64
 10  dw_06              2822 non-null   float64
 11  dw_07              2822 non-null   float64
 12  dw_08              2822 non-null   float64
 13  dw_09              2822 non-null   float64
 14  dw_10              2822 non-null   float64
 15  dw_11              2822 non-null   float64
 16  dw_12              2822 

In [4]:
df.describe()

,total_households,total_individuals,target,dw_00,dw_01,dw_02,dw_03,dw_04,dw_05,dw_06,...,pw_02,pw_03,pw_04,pw_05,pw_06,pw_07,pw_08,lat,lon,NL
count,2822.000000,2822.000000,2822.000000,2822.000000,2822.000000,2822.000000,2822.000000,2822.000000,2822.000000,2822.000000,...,2822.000000,2822.000000,2822.000000,2822.000000,2822.000000,2822.0,2822.0,2822.000000,2822.000000,2822.000000
mean,3665.281214,12869.130053,24.507554,0.712196,0.092616,0.032043,0.006057,0.008665,0.006289,0.022375,...,0.127555,0.041589,0.019655,0.011008,0.110818,0.0,0.0,-26.880740,28.666515,17.437560
std,3266.364522,9696.690518,10.294387,0.214035,0.182852,0.080253,0.019374,0.030697,0.024606,0.037127,...,0.156400,0.057331,0.032926,0.023672,0.185401,0.0,0.0,2.021279,2.373809,18.958621
min,1.000000,402.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,-32.490089,16.760022,0.000000
25%,1778.858235,7071.205695,16.751556,0.594212,0.002895,0.002407,0.000000,0.000000,0.000000,0.002716,...,0.008673,0.002099,0.000715,0.000159,0.005217,0.0,0.0,-28.569019,27.707932,3.033397
50%,2398.249935,9366.989680,24.156670,0.766841,0.010425,0.005762,0.000807,0.000607,0.000865,0.008639,...,0.069065,0.016496,0.005164,0.001459,0.025165,0.0,0.0,-26.549866,28.959679,9.205572
75%,3987.080563,14241.233015,32.226553,0.881708,0.068209,0.027913,0.002538,0.002225,0.003027,0.025218,...,0.183384,0.058626,0.025055,0.009432,0.116638,0.0,0.0,-25.572130,30.441505,26.890531
max,39684.942130,91716.746370,55.528423,0.994962,0.931490,0.951806,0.264239,0.392085,0.435912,0.412936,...,1.000000,0.327393,0.306787,0.228261,0.961523,0.0,0.0,-22.331267,32.858249,63.000000


['dw_13', 'lan_13', 'dw_12', 'pw_08', 'pw_07'] have only 0 in them --> we could drop these features

In [5]:
# drop features with only 0 in them
df = df.drop(['dw_13', 'lan_13', 'dw_12', 'pw_08', 'pw_07', 'lat', 'lon', 'ward', 'ADM4_PCODE'], axis=1)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2822 entries, 0 to 2821
Data columns (total 54 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   total_households   2822 non-null   float64
 1   total_individuals  2822 non-null   float64
 2   target             2822 non-null   float64
 3   dw_00              2822 non-null   float64
 4   dw_01              2822 non-null   float64
 5   dw_02              2822 non-null   float64
 6   dw_03              2822 non-null   float64
 7   dw_04              2822 non-null   float64
 8   dw_05              2822 non-null   float64
 9   dw_06              2822 non-null   float64
 10  dw_07              2822 non-null   float64
 11  dw_08              2822 non-null   float64
 12  dw_09              2822 non-null   float64
 13  dw_10              2822 non-null   float64
 14  dw_11              2822 non-null   float64
 15  psa_00             2822 non-null   float64
 16  psa_01             2822 

In [7]:
df.shape

(2822, 54)

In [10]:
reduced_feature_list = ['pw_00', 'psa_00', 'car_00', 'pg_03', 'NL', 'lan_00', 'pg_00', 'target']
df_red = df[reduced_feature_list]

In [11]:
# transform the data
def convert_zeros(x):
    '''
    function to convert zeros to a postive number 
    so that it can be transformed with the boxcox'''
    if x == 0.0:
        return 0.0000001
    else :
        return x


In [12]:


for col in reduced_feature_list:
    if col != 'target' and  col != 'psa_00' and col != 'car_00' and col != 'pg_00':
        df_red[col] = df_red[col].apply(convert_zeros)
        df_red[col] = boxcox(df_red[col])[0].reshape(-1,1);

/var/folders/8j/mj8c2cg9517b0ht89l57gj9m0000gn/T/ipykernel_84057/987145503.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_red[col] = df_red[col].apply(convert_zeros)
/var/folders/8j/mj8c2cg9517b0ht89l57gj9m0000gn/T/ipykernel_84057/987145503.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_red[col] = boxcox(df_red[col])[0].reshape(-1,1);


In [13]:
df_red.head()

,pw_00,psa_00,car_00,pg_03,NL,lan_00,pg_00,target
0,-0.284113,0.260191,0.273727,-2.468668,-1.053425,-0.183226,0.357455,16.773757
1,-0.982978,0.290228,0.144638,-3.780784,1.361363,-0.128422,0.698428,21.496661
2,-0.951828,0.186435,0.272220,-1.716657,-3.789711,-0.573863,0.672452,10.931425
3,-0.366754,0.281229,0.127875,-5.967212,0.782549,-0.297017,0.728014,23.119257
4,-0.249689,0.196687,0.404507,-2.126300,-3.789711,-0.870200,0.753491,13.652252


In [14]:
# Train test split with same random seed
# Defining X and y
features = df_red.columns.tolist()
features.remove('target')
X = df[features]
y = df.target

# Splitting the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=150, shuffle=True)

# Check the shape of the data sets
print("X_train:", X_train.shape)
print("y_train:", y_train.shape)
print("X_test:", X_test.shape)
print("y_test:", y_test.shape)

X_train: (2116, 7)
y_train: (2116,)
X_test: (706, 7)
y_test: (706,)


# KNN Model

In [81]:
# Import packages for KNN

from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

In [82]:
# normalize the data

from sklearn.preprocessing import MinMaxScaler

# fit scaler on training data
norm = MinMaxScaler().fit(X_train)

# transform training data
X_train_norm = norm.transform(X_train)

# transform testing data
X_test_norm = norm.transform(X_test)

In [83]:
# Train model

knn = KNeighborsRegressor(n_neighbors=5, metric='euclidean') # first try with default value of n_neighbors = 5
knn.fit(X_train_norm, (y_train))

KNeighborsRegressor(metric='euclidean')

In [84]:
# Predict target values
y_pred = knn.predict(X_test_norm)

In [85]:
# Evaluate the model
MSE = mean_squared_error(y_pred, y_test)
RMSE = np.sqrt(MSE)

print(RMSE)

4.459630875063243


In [89]:
# Train model other hyperparameters

knn2 = KNeighborsRegressor(n_neighbors=5, p=1) # first try with default value of n_neighbors = 5
knn2.fit(X_train_norm, (y_train))

KNeighborsRegressor(p=1)

In [90]:
# Predict target values and print RMSE
y_pred2 = knn2.predict(X_test_norm)

MSE = mean_squared_error(y_pred2, y_test)
RMSE = np.sqrt(MSE)

print(RMSE)

4.522615154553224


In [91]:
# try different values for n_neighbors

rmse_val = [] #to store rmse values for different k
for K in range(20):
    K = K+1
    model = KNeighborsRegressor(n_neighbors = K)

    model.fit(X_train_norm, y_train)  #fit the model
    pred=model.predict(X_test_norm) #make prediction on test set
    error = np.sqrt(mean_squared_error(y_test,pred)) #calculate rmse
    rmse_val.append(error) #store rmse values
    print('RMSE value for k= ' , K , 'is:', error)

RMSE value for k=  1 is: 5.442844470896378
RMSE value for k=  2 is: 4.891768586459219
RMSE value for k=  3 is: 4.64521356724017
RMSE value for k=  4 is: 4.522118508000427
RMSE value for k=  5 is: 4.459630875063243
RMSE value for k=  6 is: 4.387535875409072
RMSE value for k=  7 is: 4.391032458979832
RMSE value for k=  8 is: 4.370890937977278
RMSE value for k=  9 is: 4.353183699158731
RMSE value for k=  10 is: 4.362507885692046
RMSE value for k=  11 is: 4.333602589294788
RMSE value for k=  12 is: 4.3170072405426465
RMSE value for k=  13 is: 4.309139491630204
RMSE value for k=  14 is: 4.29469568700869
RMSE value for k=  15 is: 4.29747941343136
RMSE value for k=  16 is: 4.2993724893584675
RMSE value for k=  17 is: 4.304687534209672
RMSE value for k=  18 is: 4.3244860423720475
RMSE value for k=  19 is: 4.334205482309178
RMSE value for k=  20 is: 4.318221138876896


In [106]:
# Print best score
rmse_val.sort()
print("Best value: ", rmse_val[0])

Best value:  4.29469568700869


# Support Vector Machine

In [64]:
# import package for SVM Model
from sklearn.svm import SVR

In [77]:
# Train the model
clf = SVR(kernel='linear', verbose=2)
clf.fit(X_train, y_train) 

[LibSVM].........................
*....
*........................*.......................................................................*.........................................*..............................................*..............................................................................................................................................................................................................*.....................*.......................................................................................*.....................*
optimization finished, #iter = 540152
obj = -8666.818081, rho = -21.756556


SVR(kernel='linear', verbose=2)

nSV = 2078, nBSV = 2071


In [76]:
# Predict target values and print RMSE
y_pred_svm = clf.predict(X_test)

MSE = mean_squared_error(y_pred_svm, y_test)
RMSE = np.sqrt(MSE)

print(RMSE)

4.824088463407089
